In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor
## csv로 저장되어 있는 값을 pandas의 DataFrame 값으로 변환 하기
machine = pd.read_csv("machine_data.csv")
## 컬럼이 무엇이 있는지 확인
machine.columns
import missingno as msn
# 결측값의 개수를 막대 그래프로 시각화 한다.
msn.bar(machine)
# 정확성을 위해서 결측치의 값을 제거해 준다
# 결측치가 하나도 없는 것을 알 수 있다.
machine.dropna(inplace = True)
#결측치가 없는 것을 확인했기 때문에 PRP와의 상관 관계를 계산해서 변수를 선택해줘야한다.
# pandas 라이브러리에 포함된 DataFrame의 메서드인 corr() 메서드를 사용해야 한다.
# 또한 상관도를 구할 때 모든 변수들의 값은 수치화 되어야 한다.
# 모든 변수를 수치화 한다.
dums = pd.get_dummies(machine, dtype=float)
# PRP와의 상관계수 계산
prp_resulte = dums.corr()['PRP']
# 상관계수 값을 절대값으로 변환
# 변환의 이유는 간단하다. corr() 메서드가 -1~1의 값을 가져오기 때문이다.
prp_resulte= np.abs(prp_resulte)
# 상관계수를 내림차순으로 정렬
prp_resulte.sort_values(ascending=False)
# 내림차순으로 선택 후 상관 관계가 높은 features 선택
num_features = prp_resulte.sort_values(ascending=False).index[1:6].tolist()
X_train = dums[num_features]
X_test = dums[num_features]
y_train = dums['PRP']
y_test = dums['PRP']
# 앙상블 기법으로 모델 생성 및 학습  : LinearRegression, SGDRegressor, Ridge, GradientBoostingRegressor
num_transformer = Pipeline(steps=[('scaler', StandardScaler())])
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_features)])
# 개별 모델 정의
models = [
    ('lr', LinearRegression()),
    ('sgd', SGDRegressor()),
    ('ridge', Ridge(random_state=42)),
    ('gbr', GradientBoostingRegressor(random_state=42))
]
# 앙상블 모델 정의
ensemble = VotingRegressor(estimators=models)

# 모델 파이프라인 생성
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('ensemble', ensemble)])

# 하이퍼파라미터 튜닝
params_grid = {
    "ensemble__sgd__alpha": [0.0001, 0.1, 1.0],
    "ensemble__sgd__max_iter": [1000, 2000, 3000],
    "ensemble__ridge__alpha": [10, 20],
    "ensemble__gbr__n_estimators": [100, 200],
    "ensemble__gbr__learning_rate": [0.01, 0.1]
}
grid_search = GridSearchCV(model_pipeline, params_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)
# 최적 파라미터를 이용하여 최적 모델로 예측하기
# 1) 최적 파라미터 출력
# print(f'Best parameters: {grid_search.best_params_}')

# 최적 모델로 예측
best_model = grid_search.best_estimator_

# 2) 최적 모델 출력
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R^2 Score: {r2}')

# 최적 모델 성능 평가 및 시각화
plt.scatter(y_test, y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

# GridSearchCV 결과 분석
result = pd.DataFrame(grid_search.cv_results_)
result